# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [35]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import json
import pandas as pd
import numpy as np
import requests
import gmaps
import os
import time
import scipy.stats as st
from scipy.stats import linregress
import os



# Import API key
from api_keys import g_key
gmaps.configure(api_key=g_key)

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [36]:
file="../WeatherPy/output_data/cities.csv"
cities_df=pd.read_csv(file)
cities_df

,Unnamed: 0,City,Lat,Lng,Max Temp ˚F,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,Aasiaat,68.7098,-52.8699,3.60,85,100,10.78,GL,2021-04-17 02:45:42
1,1,Abbeville,50.1000,1.8333,28.99,83,11,5.61,FR,2021-04-17 02:47:31
2,2,Abhā,18.2164,42.5053,60.80,27,0,5.53,SA,2021-04-17 02:47:23
3,3,Abnūb,27.2696,31.1511,68.00,37,0,11.50,EG,2021-04-17 02:47:06
4,4,Acajutla,13.5928,-89.8275,72.00,93,99,1.99,SV,2021-04-17 02:45:26
...,...,...,...,...,...,...,...,...,...,...
575,575,Zhigansk,66.7697,123.3711,21.43,76,100,19.01,RU,2021-04-17 02:44:42
576,576,Zile,40.3031,35.8864,44.60,87,75,4.61,TR,2021-04-17 02:46:52
577,577,Örnsköldsvik,63.2909,18.7153,32.00,86,72,5.75,SE,2021-04-17 02:45:29
578,578,Āsosa,10.0667,34.5333,69.39,15,23,4.09,ET,2021-04-17 02:46:09


In [37]:
cities_df.set_index("City")
cities_df.drop(columns=["Unnamed: 0"])

,City,Lat,Lng,Max Temp ˚F,Humidity,Cloudiness,Wind Speed,Country,Date
0,Aasiaat,68.7098,-52.8699,3.60,85,100,10.78,GL,2021-04-17 02:45:42
1,Abbeville,50.1000,1.8333,28.99,83,11,5.61,FR,2021-04-17 02:47:31
2,Abhā,18.2164,42.5053,60.80,27,0,5.53,SA,2021-04-17 02:47:23
3,Abnūb,27.2696,31.1511,68.00,37,0,11.50,EG,2021-04-17 02:47:06
4,Acajutla,13.5928,-89.8275,72.00,93,99,1.99,SV,2021-04-17 02:45:26
...,...,...,...,...,...,...,...,...,...
575,Zhigansk,66.7697,123.3711,21.43,76,100,19.01,RU,2021-04-17 02:44:42
576,Zile,40.3031,35.8864,44.60,87,75,4.61,TR,2021-04-17 02:46:52
577,Örnsköldsvik,63.2909,18.7153,32.00,86,72,5.75,SE,2021-04-17 02:45:29
578,Āsosa,10.0667,34.5333,69.39,15,23,4.09,ET,2021-04-17 02:46:09


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [38]:
humidity=cities_df['Humidity'].astype(float)
humidmax=humidity.max()
humidity


0      85.0
1      83.0
2      27.0
3      37.0
4      93.0
       ... 
575    76.0
576    87.0
577    86.0
578    15.0
579    95.0
Name: Humidity, Length: 580, dtype: float64

In [39]:
location=cities_df[['Lat','Lng']]
location

,Lat,Lng
0,68.7098,-52.8699
1,50.1000,1.8333
2,18.2164,42.5053
3,27.2696,31.1511
4,13.5928,-89.8275
...,...,...
575,66.7697,123.3711
576,40.3031,35.8864
577,63.2909,18.7153
578,10.0667,34.5333


In [69]:
figure_layout = {
    'width': '800px',
    'height': '600px',
    'border': '1px solid black',
    'padding': '1px',
    'margin': '0 auto 0 auto'
}
fig=gmaps.figure(layout=figure_layout)
#heat_layer=gmaps.heatmap_layer(coordinates, weights=cities_df['Humidity'], max_intensity=humidmax, point_radius=3)
heat_layer = gmaps.heatmap_layer(location, weights=cities_df['Humidity'], 
                                 dissipating=False, max_intensity=100,
                                 point_radius=5)

fig.add_layer(heat_layer)
fig


Figure(layout=FigureLayout(border='1px solid black', height='600px', margin='0 auto 0 auto', padding='1px', wi…

In [55]:
cities_df.count()

Unnamed: 0     580
City           580
Lat            580
Lng            580
Max Temp ˚F    580
Humidity       580
Cloudiness     580
Wind Speed     580
Country        573
Date           580
dtype: int64

In [56]:
weather_filter_df=cities_df.loc[(cities_df['Max Temp ˚F'] > 70) & (cities_df['Max Temp ˚F'] < 80) & (cities_df['Wind Speed'] < 10) & (cities_df['Cloudiness'] == 0), :]
weather_filter_df = weather_filter_df.dropna(how='any')
weather_filter_df.reset_index(inplace=True)
del weather_filter_df['index']
del weather_filter_df["Unnamed: 0"]
weather_filter_df.tail()


,City,Lat,Lng,Max Temp ˚F,Humidity,Cloudiness,Wind Speed,Country,Date
7,Saint-Philippe,-21.3585,55.7679,77.00,83,0,3.44,RE,2021-04-17 02:47:02
8,Saint-Pierre,-21.3393,55.4781,77.00,83,0,3.44,RE,2021-04-17 02:46:57
9,Salalah,17.0151,54.0924,77.00,73,0,5.75,OM,2021-04-17 02:45:14
10,Santa Cruz de la Sierra,-17.8000,-63.1667,73.08,77,0,5.08,BO,2021-04-17 02:43:08
11,Santa Fe,-31.6333,-60.7000,72.00,85,0,1.99,AR,2021-04-17 02:46:51


In [57]:
weather_filter_df.set_index("City")
weather_filter_df.count()

City           12
Lat            12
Lng            12
Max Temp ˚F    12
Humidity       12
Cloudiness     12
Wind Speed     12
Country        12
Date           12
dtype: int64

### Hotel Map

* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [58]:
# Store into variable named hotel_df.
hotel_df=weather_filter_df
# Add a "Hotel Name" column to the DataFrame.
hotel_df["Hotel Name"] = ""
hotel_df

,City,Lat,Lng,Max Temp ˚F,Humidity,Cloudiness,Wind Speed,Country,Date,Hotel Name
0,Atar,20.5169,-13.0499,71.62,29,0,9.69,MR,2021-04-17 02:44:36,
1,Baraguá,21.6967,-78.6328,73.92,79,0,2.48,CU,2021-04-17 02:46:59,
2,Coahuayana Viejo,18.7333,-103.6833,70.54,78,0,1.57,MX,2021-04-17 02:45:28,
3,Córdoba,-31.4135,-64.1811,72.00,94,0,6.91,AR,2021-04-17 02:42:35,
4,Oussouye,12.4850,-16.5469,75.20,78,0,2.30,SN,2021-04-17 02:47:07,
5,Rohri,27.6922,68.8951,71.60,64,0,6.91,PK,2021-04-17 02:45:25,
6,Saint-Joseph,-21.3667,55.6167,77.00,83,0,3.44,RE,2021-04-17 02:47:22,
7,Saint-Philippe,-21.3585,55.7679,77.00,83,0,3.44,RE,2021-04-17 02:47:02,
8,Saint-Pierre,-21.3393,55.4781,77.00,83,0,3.44,RE,2021-04-17 02:46:57,
9,Salalah,17.0151,54.0924,77.00,73,0,5.75,OM,2021-04-17 02:45:14,


In [59]:
hotellist=[]
# Set parameters to search for hotels with 5000 meters.
params = {
        "radius": 5000,
        "types": "lodging",
        "key": g_key
}

base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

for index, row in hotel_df.iterrows():
    lat=row["Lat"]
    lng=row["Lng"]
    if row["Hotel Name"]=="":
        params["location"]=f"{lat},{lng}"
        name=requests.get(base_url, params=params).json()
        hotel_df.loc[index, "Hotel Name"] = name["results"][0]["name"]
hotel_df

,City,Lat,Lng,Max Temp ˚F,Humidity,Cloudiness,Wind Speed,Country,Date,Hotel Name
0,Atar,20.5169,-13.0499,71.62,29,0,9.69,MR,2021-04-17 02:44:36,Oumou Elghoura
1,Baraguá,21.6967,-78.6328,73.92,79,0,2.48,CU,2021-04-17 02:46:59,Hotel Itabo
2,Coahuayana Viejo,18.7333,-103.6833,70.54,78,0,1.57,MX,2021-04-17 02:45:28,El rranchito michuacan de ocampos
3,Córdoba,-31.4135,-64.1811,72.00,94,0,6.91,AR,2021-04-17 02:42:35,Hotel Sussex
4,Oussouye,12.4850,-16.5469,75.20,78,0,2.30,SN,2021-04-17 02:47:07,Auberge du Routard
5,Rohri,27.6922,68.8951,71.60,64,0,6.91,PK,2021-04-17 02:45:25,shaikh house rohri
6,Saint-Joseph,-21.3667,55.6167,77.00,83,0,3.44,RE,2021-04-17 02:47:22,"""Plantation Bed and Breakfast"
7,Saint-Philippe,-21.3585,55.7679,77.00,83,0,3.44,RE,2021-04-17 02:47:02,"Chambres d'hôte ""La Trinité"""
8,Saint-Pierre,-21.3393,55.4781,77.00,83,0,3.44,RE,2021-04-17 02:46:57,Lindsey Hôtel
9,Salalah,17.0151,54.0924,77.00,73,0,5.75,OM,2021-04-17 02:45:14,HAMDAN PLAZA HOTEL SALALAH


In [65]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [70]:
# Add marker layer ontop of heat map
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)

fig.add_layer(marker_layer)
fig

Figure(layout=FigureLayout(border='1px solid black', height='600px', margin='0 auto 0 auto', padding='1px', wi…